In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import numpy as np

def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()

In [ ]:
model_name="kanhatakeyama/01b_model_30b_token"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto",
    torch_dtype=torch.float16
)


In [ ]:
text_list=[
    "生八つ橋のタグまとめ|エキサイトブログ\n生八つ橋のタグまとめ\n「生八つ橋」のタグがついている新着記事と人気記事をまとめました。エキサイトブログには生八つ橋に関連するブログ(日記、記録、写真、レビュー、噂、まとめ)がたくさん投稿されています。\n「生八つ橋」タグの記事(4)\n生八つ橋いろいろ\n京都旅行のお土産(我が家用)に色々な生八つ橋を買ってきました。我が家はみんな八つ橋ファンなのです～。地元の方達や京都を案内してくれたYさんは「八つ橋なんてもう何年も食べたことないわ～～～」と仰っていましたが、いやいや、美味しいですよ～～!私は大好きです!まぁ確かに私は東京出身ですが、雷おこしや人形焼きは食べませんからね～。\n2020/03/06 23:54-NYの小さな灯り～ヘアメイク日記～\n黒糖きな粉\n冬限定の生八つ橋「ふゆおたべ」\n日本から戻る時に大抵空港で生八つ橋か信玄餅のどちらかを必ず買って帰ってくるのですが、今回は珍しいバージョンを見かけたので生八つ橋を買う事にしました。それがこちら、冬限定の生八つ橋「ふゆおたべ」です。切り絵のデザインも素敵ですよね^^お味は黒豆と栗きんとんの二種類。\n2019/03/06 00:30-NYの小さな灯り～ヘアメイク日記～\n【株式会社美十】ショコラこたべ\n日帰り京都旅行、あまり時間がなかったのでお土産はすべて京都駅周辺で済ませてしまいました。が、数年ぶりに訪れた京都ではお土産にも大きな変化が...!代表的な京都土産のひとつ、生八つ橋ってあのニッキの味わいがなんか苦手...っていう方も多いと思うんですけど、今はそういう定番以外にもイマドキ風な生八つ橋がすごくいっぱいあって驚きました!\n大学生の孫が、部活の全国へ大会で、京都へ4日間行ってきたそうです。おみやげに、おたべとお茶を買ってきました。来年は、福井県だそうです。まぁ!アルバイトをしているからいいけれど、交通費がかかりそう!私は八十八を見て、てっきり八つ橋かと思って開けて見たらお茶でした。ちゃんと、考えて買ってきたのでした。我が家の飲んでいるお茶は、賄い茶、たくさん飲むので、これで十分です｡",
    "fjalkjfepiwofe",
    "This is the model card of a 🤗 transformers model that has been pushed on the Hub. This model card has been automatically generated.",
]

In [ ]:
for txt in text_list:
    ppl=perplexity(model,tokenizer,txt)
    print(ppl,txt.replace("\n",""))

In [ ]:
from datasets import load_dataset
#mc4の読み込み
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)

dataset=mc4_dataset

In [ ]:
import re
for i,record in enumerate(dataset):
    txt=record["text"]
    print("-----")

    for line in re.split(r'[。\\n]+', txt):
        if line=="":
            continue
        line=line[:2000]
        ppl=perplexity(model,tokenizer,line)
        print(ppl,line.replace("\n",""))

    if i>15:
        break

In [ ]:
len(line)